In [1]:
import moyra as ma
import moyra.elements as ele
import moyra.forces as ef
import sympy as sym
import sympy.physics.mechanics as me
import warnings
import matplotlib.cbook

warnings.filterwarnings("ignore",category=matplotlib.cbook.mplDeprecation)
me.mechanics_printing()

## Rayleigh Ritz Beam Model

In this example we will generate the EoMs for a cantilever beam fixed at the root, using assumed shapes.

The shape functions used will be polynomials of order n (for bending) and m (for torsion)

In [21]:
b_modes = 3
t_modes = 2
qs = b_modes + t_modes 

p = ma.DynamicModelParameters(qs)

## Create some global parameters
p.c = ma.ModelSymbol(value = 1.8,string = 'c') # chord of wing
p.s = ma.ModelSymbol(value = 12,string = 's') # total semi-span of wing
p.rho = ma.ModelSymbol(value = 20,string = 'rho') # mass per unit area (kg/m^2)

# Inner Wing stiffness Properties
p.EI = ma.ModelSymbol(value = 1e7,string = 'EI') # Bending stiffness for the wing
p.GJ = ma.ModelSymbol(value = 1e6,string = 'GJ') # Torsional Stiffness for the wing

# Location of flexural axes
p.e = ma.ModelSymbol(value = 0,string = 'e')
p.x_f = sym.Rational(1,4)*p.c + p.e*p.c

# Symbols to translate along inner wing
p.y_0 = sym.Symbol('y_0') # inner wing y chord
p.x_0 = sym.Symbol('x_0') # inner wing x chord

# Factor to change the size of joint values
p.eta = ma.ModelMatrix(value = [1]*qs,length=qs,string='eta') # a scale factor for each shape function (to aid numerical stability)
p.g = ma.ModelSymbol(value = 9.81,string='g')

In [22]:
#get shape functions for main wing
z_0,tau_0 = ele.FlexiElement.ShapeFunctions_BN_TM(b_modes,t_modes,p.q,p.y_0,p.x_0,p.x_f,0,factor=1)
#z_0,tau_0 = ele.FlexiElement.ShapeFunctions_BN_TM(b_modes,t_modes,p.q,p.y_0,p.x_0,p.x_f,0,factor=p.eta)
wing_bend = sym.atan(z_0.diff(p.y_0).subs({p.x_0:p.x_f,p.y_0:p.s}))

#define wing refernce frames
wing_root_frame = ma.HomogenousTransform()
wing_frame = wing_root_frame.Translate(p.x_0,p.y_0,z_0)
wing_flexural_frame = wing_frame.msubs({p.x_0:p.x_f})

#Create Element Mass Matriceis
M_wing = ele.MassMatrix(p.rho)

#Create wing element
wing_ele = ele.FlexiElement(wing_root_frame,M_wing,p.x_0,p.y_0,z_0,p.c,p.s,p.x_f,p.EI,p.GJ,gravityPot=True) 

# Create the Symbolic Model
sm = ma.SymbolicModel.FromElementsAndForces(p,[wing_ele])

In [23]:
sm.M

⎡           5                    6                    7                 2     
⎢      c⋅ρ⋅s                c⋅ρ⋅s                c⋅ρ⋅s                 c ⋅e⋅ρ⋅
⎢      ──────               ──────               ──────                ───────
⎢        5                    6                    7                       4  
⎢                                                                             
⎢           6                    7                    8                 2     
⎢      c⋅ρ⋅s                c⋅ρ⋅s                c⋅ρ⋅s                 c ⋅e⋅ρ⋅
⎢      ──────               ──────               ──────                ───────
⎢        6                    7                    8                       5  
⎢                                                                             
⎢           7                    8                    9                 2     
⎢      c⋅ρ⋅s                c⋅ρ⋅s                c⋅ρ⋅s                 c ⋅e⋅ρ⋅
⎢      ──────               ──────               ───

In [24]:
sm.f

⎡                                               3  ⎤
⎢        3            2                  c⋅g⋅ρ⋅s   ⎥
⎢  8⋅EI⋅s ⋅q₂ + 6⋅EI⋅s ⋅q₁ + 4⋅EI⋅s⋅q₀ + ────────  ⎥
⎢                                           3      ⎥
⎢                                                  ⎥
⎢                                                4 ⎥
⎢       4             3            2      c⋅g⋅ρ⋅s  ⎥
⎢18⋅EI⋅s ⋅q₂ + 12⋅EI⋅s ⋅q₁ + 6⋅EI⋅s ⋅q₀ + ──────── ⎥
⎢                                            4     ⎥
⎢                                                  ⎥
⎢        5                                        5⎥
⎢144⋅EI⋅s ⋅q₂          4            3      c⋅g⋅ρ⋅s ⎥
⎢──────────── + 18⋅EI⋅s ⋅q₁ + 8⋅EI⋅s ⋅q₀ + ────────⎥
⎢     5                                       5    ⎥
⎢                                                  ⎥
⎢                           ⎛ 2          2    ⎞    ⎥
⎢       2                 2 ⎜c ⋅e⋅g⋅ρ   c ⋅g⋅ρ⎟    ⎥
⎢   GJ⋅s ⋅q₄ + GJ⋅s⋅q₃ + s ⋅⎜──────── - ──────⎟    ⎥
⎢                           ⎝   2         8   

In [25]:
sm.to_matlab_file(p,'')